# AACT

В `aact.csv` находится часть записей из базы AACT, которая использовалась в предыдущей  неделе. Попробуем поработать с ней только из bash.

ссылка на колаб -- https://colab.research.google.com/drive/1ZuK-qDkO8wjOS6Uva7R-0BUY7RXinjHh?usp=sharing

In [1]:
! head aact.csv

,nct_id,study_type,acronym,baseline_population,brief_title,official_title,overall_status
0,NCT04452825,Observational,,,A New Psychotherapy Intervention for Older Cancer Patients,Cancer and Aging Reflections for Elders (CARE): A Geriatric-Specific Psychotherapy for Older Adults With Cancer,"Active, not recruiting"
1,NCT04452812,Interventional,PROMETEO,,Statistical and Epidemiological Study Based on the Use of Convalescent Plasma for the Management of Patients With COVID-19,"Pilot Clinical, Statistical and Epidemiological Study on Efficacy and Safety of Convalescent Plasma for the Management of Patients With COVID-19",Not yet recruiting
2,NCT04452799,Interventional,,,Hesperidin and Diosmin for Treatment of COVID-19,Randomized Double-blind Controlled Parallel Study of (Hesperidin and Diosmin Mixture) for Treatment of COVID-19 Newly Diagnosed Patients in Egypt,Not yet recruiting
3,NCT04452786,Interventional,,,Hormonal and Metabolic Aspects of Endoscopic Sleeve Gastroplasty,Hormonal and Met

Посмотрим, какие вообще есть `study_type`

* Отрезаем заголовок
* Разбиваем по запятой
* Сортируем и считаем уникальные

In [2]:
! cat aact.csv | tail -n +2 | cut -d"," -f3 | sort | uniq

Expanded Access
Interventional
N/A
Observational
Observational [Patient Registry]


cat - просто читает файл (https://losst.ru/komanda-cat-linux)

| - передает выход предыдущей команды как вход на следующую 

tail - смотрим последние строки (если бы мы передали -n 10 -- было бы последние 10 строк, а так мы смотрим до 2й строки) (https://losst.ru/komanda-tail-linux)

cut - вырезает элемент -- с помощью d выбираем разделитель, а с помощью f -- номер нужного нам столбца (https://losst.ru/komanda-cut-linux)

sort - сортировка 

uniq - выибираем уникальные

Уберем N\A значения - не хотим их видеть в выборке

In [3]:
! cat aact.csv | wc -l

1001


wc - word count, с параметром l считаем строки (https://losst.ru/komanda-wc-v-linux) 

In [4]:
! cat aact.csv | tail -n +2 | sed -e "/N\/A/"d | wc -l

996


sed - позволяет редактировать файл, не открывая его (e просто идет перед командами, а d удаляет регулярные выражения выбранные) (https://losst.ru/komanda-sed-linux)

In [5]:
! cat aact.csv | tail -n +2 | sed -e "/N\/A/"d | cut -d"," -f3 | sort | uniq

Expanded Access
Interventional
Observational
Observational [Patient Registry]


Отлично, сохраним этот результат в промежуточных файл

In [6]:
! cat aact.csv | tail -n +2 | sed -e "/N\/A/"d > aact-fillna.csv

Предположим, что мы хотим составить набор данных для алгоритма машинного обучения, который бы предсказывал тип исследования по его описанию.

Для этого нам необходимо оставить только текст и значение study_type, перемешать все данные и разделить их на два набора данных - один train для обучения машины, другой test для проверки корректности работы в соотношении 70\30.

In [ ]:
! cat aact-fillna.csv | cut -d"," -f3,6,7 | head

In [7]:
! cat aact-fillna.csv | wc -l 

996


In [8]:
! echo $(( $(cat aact-fillna.csv | wc -l) * 70 / 100 ))

697


echo - просто выводит результат (https://losst.ru/komanda-echo-v-linux)

In [9]:
%%bash

export TRAIN_SIZE=$(( $(cat aact-fillna.csv | wc -l) * 70 / 100 ))
echo $TRAIN_SIZE

cat aact-fillna.csv | cut -d"," -f3,6,7 | shuf > shuffled-aact.csv

cat shuffled-aact.csv | head -n $TRAIN_SIZE > train-aact.csv
cat shuffled-aact.csv | tail -n +$(($TRAIN_SIZE+1)) > test-aact.csv

697


shuf - позволяет делать случайные перестановки

In [10]:
! cat train-aact.csv | wc -l

697


In [11]:
! cat test-aact.csv | wc -l

299


Команда `sort` также имеет возможность сортировать поток не по целой строке, а по определенной ее части.
Ключ `-t` указывает разделитель строки , а ключ `-k` указывает номер секции (колонки) по которой необходимо сортировать. 

In [12]:
! cat aact.csv | sort -t ',' -k2 | head  # сортируем по nct id

628,NCT00004323,Interventional,,,Phase II Study of Bone Marrow Transplantation Using Related Donors in Patients With Aplastic Anemia,,Completed
567,NCT00004324,Observational,,,Developmental Phenomenology of Obsessive Compulsive Disorder and Tourette Syndrome in Children and Adolescents,,Completed
337,NCT00004378,Interventional,,,Stem Cell Transplantation (SCT) for Genetic Diseases,,Completed
228,NCT00004380,Interventional,,,Phase II Study of Recombinant Relaxin for Progressive Systemic Sclerosis,,Completed
179,NCT03869658,Observational,,,Food Talk: A New App for Daily Food Monitoring,Food Talk: A New App for Daily Food Monitoring,Completed
82,NCT03895034,N/A,,,[Trial of device that is not approved or cleared by the U.S. FDA],[Trial of device that is not approved or cleared by the U.S. FDA],Withheld
91,NCT03914872,Expanded Access,,,Expanded Access Neoantigen Vaccine in Solid Tumors,Expanded Access Neoantigen Vaccine in Solid Tumors,Temporarily not available
69,NCT04002960,N/A,,,[Trial o

# COVID

Поэкспериментируем теперь с базой covid, которая опять же была на прошлой неделе.

In [13]:
! head covid.json

{
    "measures": [
        {
            "uid": 4,
            "country_iso2": "AF",
            "country_iso3": "AFG",
            "country_code": 4,
            "country": "Afghanistan",
            "combined_name": "Afghanistan",
            "population": 38928341,


Давайте узнаем общее количество зараженных, согласно данным

In [14]:
! apt-get install -y jq

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libjq1 libonig4
The following NEW packages will be installed:
  jq libjq1 libonig4
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 276 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libonig4 amd64 6.7.0-1 [119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libjq1 amd64 1.5+dfsg-2 [111 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 jq amd64 1.5+dfsg-2 [45.6 kB]
Fetched 276 kB in 8s (36.0 kB/s)
Selecting previously unselected package libonig4:amd64.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../lib

In [15]:
! cat covid.json | jq '.measures[0] | keys'

[
  "combined_name",
  "confirmed",
  "country",
  "country_code",
  "country_iso2",
  "country_iso3",
  "deaths",
  "loc",
  "population",
  "recovered",
  "uid"
]


jq - позволяет работать с json

In [16]:
! cat covid.json | jq '.measures[].confirmed' | head

0
0
0
0
0
0
0
0
0
0


In [17]:
! cat covid.json | jq '.measures[].confirmed' | awk '{sum += $1} END {print sum}'

2339039


awk - позволяет выполнять над файлом построчные операции, подробнее читайте по ссылке (https://losst.ru/ispolzovanie-awk-v-linux)

Может точно также подсчитать среднее

In [18]:
! cat covid.json | jq '.measures[].confirmed' | awk '{sum += $1; count += 1} END { print sum / count }'

2339.04


С медианой придется идти немного другим путем

In [19]:
%%bash

export MEDIAN_INDEX=$(( $(cat covid.json | jq '.measures[].confirmed' | wc -l) / 2 ))
echo $MEDIAN_INDEX

cat covid.json | jq '.measures[].confirmed' | sort -n | sed -n -e "$MEDIAN_INDEX"p

500
61


# Word count

Давайте попробуем подсчитать количество раз, которое встречается каждое слово в документе. В качестве самого документа со словами будем использовать заголовки из aact

In [20]:
! cat aact.csv | tail -n +2 | cut -d"," -f6,7 | head  # получаем только тексты

A New Psychotherapy Intervention for Older Cancer Patients,Cancer and Aging Reflections for Elders (CARE): A Geriatric-Specific Psychotherapy for Older Adults With Cancer
Statistical and Epidemiological Study Based on the Use of Convalescent Plasma for the Management of Patients With COVID-19,"Pilot Clinical
Hesperidin and Diosmin for Treatment of COVID-19,Randomized Double-blind Controlled Parallel Study of (Hesperidin and Diosmin Mixture) for Treatment of COVID-19 Newly Diagnosed Patients in Egypt
Hormonal and Metabolic Aspects of Endoscopic Sleeve Gastroplasty,Hormonal and Metabolic Aspects of Endoscopic Sleeve Gastroplasty
Clinical Trial to Evaluate the Efficacy of Food Supplement Manremyc® Against SARS- COV-2 Infection (COVID-19) in Healthcare Workers,"Double-blind
"Effects of Ten-Weeks Progressive Resistance Training on Neuromuscular Performance, Mobility
Artificial Induction of Labour in Full-term Singleton Pregnancy : Comparative Randomized Trial of Two Strategies,Artificial In

In [21]:
! cat aact.csv | tail -n +2 | cut -d"," -f6,7 | sed -e "s/\s\+/\\n/g"  | head  # заменяем все пробелы на перенос строк

A
New
Psychotherapy
Intervention
for
Older
Cancer
Patients,Cancer
and
Aging


In [22]:
! cat aact.csv | tail -n +2 | cut -d"," -f6,7 | sed -e "s/\s\+/\\n/g" | sort | uniq -c # сортируем и схлопываем

Выходные данные были обрезаны до нескольких последних строк (5000).
      1 AspivixTM"
      1 ASPS,"A
      1 ASS
      1 Assay
      1 Assay,"A
      1 Assertive
     21 Assess
     11 Assessing
     26 Assessment
      2 Assist
      2 Assistance
      3 Assisted
      2 Assisting
      1 Assiut
     13 Associated
      3 Association
      1 "Association
      5 Asthma
      2 ""Asthma
      1 Asthma,AIM2ACT:
      5 Asthmatic
      5 Asymptomatic
      2 Asynchronies
      1 Asynchrony
      1 Asynchrony,Analysis
     30 at
      1 At
      1 Atazanavir
      3 Atezolizumab
      2 Athletes
      1 Athletes."
      1 Atomized
      6 Atopic
      5 Atrial
      1 Atrophy,
      1 Atrophy,"TRACK-MSA:
      3 Attacks
      1 Attending
      2 Attention
      1 Attentional
      4 Attenuation
      1 Attitude
      1 Attitudes
      1 Atypical
      1 Audio
      1 Audio,An
      5 Autism
      1 Autism:
      2 Auto-Antibody
      1 Autogenic
      8 Autologous
      1 Autologous-Der

Теперь хотим узнать топ 20 самых популярных слов в тексте

In [23]:
! cat aact.csv | tail -n +2 | cut -d"," -f6,7 | sed -e "s/\s\+/\\n/g" | sort | uniq -c > word-count.txt

In [24]:
# -k1 = первое поле, n = сравниваем как числа, r - в обратном порядке
! cat word-count.txt | sort -t ' ' -k1nr | head -n 20  

   1457 of
    978 in
    710 and
    543 the
    534 With
    349 for
    334 Study
    324 Patients
    278 to
    249 on
    207 a
    178 A
    153 Treatment
    132 COVID-19
    130 Trial
    103 Randomized
     97 After
     90 During
     89 Clinical
     84 Phase


В файле measures.json содержатся наблюдения о новых заболевших коронавирусом. Каждая запись соответствует одному дню наблюдения. Для каждой записи в поле country указана страна , где производились измерения. В поле data указаны сами измерения в формате csv. В каждой строке этой таблицы csv лежат два числа - заболевшие женщины и заболевшие мужчины. Каждая строка относится к своему региону в стране, где производились измерения.

Необходимо выяснить, в какой стране было наибольшее количество дней измерений.
Для этой страны необходимо подсчитать суммарное количество заболевших во все дни наблюдений.
Это число (итоговое количество заболевших) - ответ на задачу

Подсказка: для того, чтобы восстановить привычный вид csv таблицы, воспользуйтесь sed, чтобы подправить переносы строк и кавычки. Можно использовать любые средства, описанные в стандартной документации инструментов, которые мы разбирали.

In [25]:
! cat measures.json

{"measures": [{"country": "USA", "data": "6,11\n7,13\n6,15\n6,14\n6,14\n6,12\n6,14\n7,13\n6,11\n6,12\n6,12\n6,11\n6,12\n6,12\n6,11\n7,13\n6,12\n6,14\n7,13\n6,12\n6,14\n6,14\n6,9\n6,12\n6,14\n6,12\n7,14\n6,13\n6,10\n6,11\n6,10\n6,12\n6,12\n7,13\n7,14\n6,12\n6,14\n6,12\n6,13\n6,10\n6,12\n6,12\n6,13\n6,13\n6,10\n6,12\n6,14\n6,11\n6,14\n6,13\n6,14\n6,12\n7,12\n6,11\n6,12\n7,14\n7,15\n6,12\n6,11\n6,13\n6,12\n6,13\n6,12\n6,12\n7,13\n6,10\n6,14\n6,13\n6,10\n6,12\n6,12\n7,13\n6,14\n6,10\n6,12\n6,12\n6,13\n6,13\n6,11\n6,12\n6,13\n6,12\n7,15\n6,12\n6,12\n6,13\n6,14\n7,13\n7,12\n6,13\n6,13\n7,14\n7,14\n6,11\n7,13\n7,13\n6,12\n6,12\n6,12\n6,11"}, {"country": "Canada", "data": "6,10\n6,13\n6,13\n6,9\n6,11\n6,12\n6,12\n6,12\n6,12\n6,12\n6,12\n7,12\n7,13\n6,11\n7,12\n6,12\n6,13\n6,13\n6,11\n6,13\n6,11\n6,10\n6,12\n6,13\n6,10\n6,12\n6,10\n6,11\n6,14\n7,13\n6,13\n7,13\n6,11\n6,12\n7,12\n6,13\n6,11\n6,12\n7,15\n6,13\n6,13\n6,11\n6,12\n6,12\n6,10\n6,11\n6,13\n6,11\n6,13\n6,13\n6,12\n6,13\n6,12\n6,14\n7,1

In [26]:
! cat measures.json | jq '.measures' 

[
  {
    "country": "USA",
    "data": "6,11\n7,13\n6,15\n6,14\n6,14\n6,12\n6,14\n7,13\n6,11\n6,12\n6,12\n6,11\n6,12\n6,12\n6,11\n7,13\n6,12\n6,14\n7,13\n6,12\n6,14\n6,14\n6,9\n6,12\n6,14\n6,12\n7,14\n6,13\n6,10\n6,11\n6,10\n6,12\n6,12\n7,13\n7,14\n6,12\n6,14\n6,12\n6,13\n6,10\n6,12\n6,12\n6,13\n6,13\n6,10\n6,12\n6,14\n6,11\n6,14\n6,13\n6,14\n6,12\n7,12\n6,11\n6,12\n7,14\n7,15\n6,12\n6,11\n6,13\n6,12\n6,13\n6,12\n6,12\n7,13\n6,10\n6,14\n6,13\n6,10\n6,12\n6,12\n7,13\n6,14\n6,10\n6,12\n6,12\n6,13\n6,13\n6,11\n6,12\n6,13\n6,12\n7,15\n6,12\n6,12\n6,13\n6,14\n7,13\n7,12\n6,13\n6,13\n7,14\n7,14\n6,11\n7,13\n7,13\n6,12\n6,12\n6,12\n6,11"
  },
  {
    "country": "Canada",
    "data": "6,10\n6,13\n6,13\n6,9\n6,11\n6,12\n6,12\n6,12\n6,12\n6,12\n6,12\n7,12\n7,13\n6,11\n7,12\n6,12\n6,13\n6,13\n6,11\n6,13\n6,11\n6,10\n6,12\n6,13\n6,10\n6,12\n6,10\n6,11\n6,14\n7,13\n6,13\n7,13\n6,11\n6,12\n7,12\n6,13\n6,11\n6,12\n7,15\n6,13\n6,13\n6,11\n6,12\n6,12\n6,10\n6,11\n6,13\n6,11\n6,13\n6,13\n6,12\n6,13\n6,

Теперь посмотрим на список стран в каждом дне измерений и посчитаем уникальное вхождение каждой страны

In [28]:
! cat measures.json | jq '<YOUR CODE HERE>' | sort | uniq -c

     74 "Canada"
     95 "Russia"
     81 "USA"


Значит Россия на 1 месте по количеству измерений 

<YOUR CODE HERE>

In [50]:
! cat measures.json | jq '<YOUR CODE HERE> | select(<YOUR CODE HERE>)'

{
  "country": "Russia",
  "data": "6,12\n6,11\n7,13\n6,12\n6,12\n6,14\n6,11\n6,14\n6,14\n7,15\n6,13\n6,13\n6,10\n6,11\n6,13\n6,13\n6,13\n6,11\n6,11\n6,13\n6,10\n6,13\n7,13\n6,11\n6,11\n6,12\n6,12\n6,11\n6,12\n6,11\n6,12\n6,14\n6,14\n7,11\n7,13\n6,13\n6,11\n6,11\n6,14\n6,14\n6,13\n6,14\n6,13\n6,13\n6,11\n6,11\n6,13\n6,12\n6,12\n6,8\n7,13\n6,14\n6,12\n6,11\n6,12\n6,12\n6,12\n6,11\n6,12\n6,11\n7,12\n6,12\n6,10\n6,10\n6,13\n6,12\n6,12\n7,13\n7,12\n7,13\n6,11\n6,12\n6,10\n6,11\n6,13\n6,11\n6,12\n6,12\n6,12\n6,12\n6,13\n7,12\n6,12\n6,10\n6,13\n6,10\n6,14\n6,15\n7,13\n6,11\n6,12\n6,11\n6,13\n6,9\n6,13\n6,11\n6,13\n6,13\n6,12\n6,14"
}
{
  "country": "Russia",
  "data": "7,15\n6,14\n6,14\n6,11\n6,12\n6,11\n6,11\n7,13\n6,13\n6,13\n6,11\n6,12\n6,12\n7,14\n7,11\n6,13\n6,13\n6,12\n6,12\n6,11\n6,12\n6,14\n6,12\n6,10\n7,13\n6,10\n7,14\n6,8\n7,14\n6,13\n6,13\n6,11\n6,11\n6,9\n6,11\n6,12\n6,14\n6,12\n7,13\n6,12\n6,13\n6,12\n6,12\n6,13\n6,12\n6,13\n7,14\n6,13\n6,12\n7,14\n6,10\n6,12\n6,11\n7,13\n6,14\n

Дальше сами

In [ ]:
<YOUR CODE HERE>